In [ ]:
!pip install -U albumentations opencv-python numpy tqdm

In [ ]:
!unzip -q data.zip -d /content/

In [ ]:
import albumentations as A
import cv2
import os
import numpy as np
from tqdm import tqdm
from glob import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
BASE_DIR = '/content/drive/MyDrive/data'
INPUT_IMAGES_DIR = os.path.join(BASE_DIR, 'images')
INPUT_LABELS_DIR = os.path.join(BASE_DIR, 'labels')
OUTPUT_DIR = os.path.join(BASE_DIR, 'augmented')
OUTPUT_IMAGES_DIR = os.path.join(OUTPUT_DIR, 'images')
OUTPUT_LABELS_DIR = os.path.join(OUTPUT_DIR, 'labels')
os.makedirs(OUTPUT_IMAGES_DIR, exist_ok=True)
os.makedirs(OUTPUT_LABELS_DIR, exist_ok=True)
AUGMENTATIONS_PER_IMAGE = 2


In [ ]:

# Augmentation
transform = A.Compose(
    [
        #Geometric transformation
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(
            shift_limit=0.0625,
            scale_limit=0.15,
            rotate_limit=180,
            p=0.9
        ),

        # pixel transformation
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.8),
        # color transformation
        A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=40, val_shift_limit=40, p=0.7),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.5), # noise
        A.Blur(blur_limit=3, p=0.3), # blur

        # randomly apply effects
        A.OneOf([
            A.Spatter(p=0.5), #dust/spots
            A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), p=0.5), # contrast
        ], p=0.4)
    ],
    bbox_params=A.BboxParams(
        format='yolo',
        label_fields=['class_labels'],
        min_visibility=0.1
    )
)


In [ ]:
def load_yolo_data(label_path):
    bboxes = []
    class_labels = []

    if not os.path.exists(label_path):
        return bboxes, class_labels

    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) == 5:
                class_id = int(parts[0])
                bbox = [float(p) for p in parts[1:]]
                bboxes.append(bbox)
                class_labels.append(class_id)
    return bboxes, class_labels

def save_yolo_data(label_path, transformed_bboxes, transformed_labels):
    with open(label_path, 'w') as f:
        for bbox, class_label in zip(transformed_bboxes, transformed_labels):
            line = f"{class_label} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n"
            f.write(line)

In [ ]:
#Get images
image_files = glob(os.path.join(INPUT_IMAGES_DIR, '*.jpg'))

for image_path in tqdm(image_files, desc="Processing Images"):
    # file names
    base_name = os.path.basename(image_path)
    file_name_no_ext, ext = os.path.splitext(base_name)
    label_path = os.path.join(INPUT_LABELS_DIR, file_name_no_ext + '.txt')

    # load data
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # convert to RGB
    bboxes, class_labels = load_yolo_data(label_path)

    for i in range(AUGMENTATIONS_PER_IMAGE):
        # apply transformation
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        transformed_labels = transformed['class_labels']
        aug_image_name = f"{file_name_no_ext}_aug{i}{ext}"
        aug_label_name = f"{file_name_no_ext}_aug{i}.txt"
        output_image_path = os.path.join(OUTPUT_IMAGES_DIR, aug_image_name)
        output_label_path = os.path.join(OUTPUT_LABELS_DIR, aug_label_name)
        cv2.imwrite(output_image_path, cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR))
        save_yolo_data(output_label_path, transformed_bboxes, transformed_labels)

print("\nAugmentation Complete")